In [1]:
import time
import numpy as np

from coppeliasim_zmqremoteapi_client import RemoteAPIClient 

def Rz(theta):
  
    return np.array([[ np.cos(theta), -np.sin(theta), 0 ],
                      [ np.sin(theta), np.cos(theta) , 0 ],
                      [ 0            , 0             , 1 ]])

# Holonomic control

In [2]:
try:
    # Connect to the CoppeliaSim server
    client = RemoteAPIClient()
    sim = client.require("sim")
    sim.setStepping(True)

    # Handle para o ROBÔ
    robotname = 'robotino'
    robotHandle = sim.getObject('/' + robotname)
    
    # Handle para as juntas das RODAS
    wheel1 = sim.getObject('/' + robotname + '/wheel0_joint')
    wheel2 = sim.getObject('/' + robotname + '/wheel1_joint')
    wheel3 = sim.getObject('/' + robotname + '/wheel2_joint')
              
    # Dados Robotino
    L = 0.135   # Metros
    r = 0.040   # Metros  
    
    # Cinemática Direta
    Mdir = np.array([[-r/np.sqrt(3), 0, r/np.sqrt(3)], [r/3, (-2*r)/3, r/3], [r/(3*L), r/(3*L), r/(3*L)]])
           
    # Goal configuration (x, y, theta)    
    qgoal = np.array([3, 3, np.deg2rad(90)])
    #qgoal = np.array([3, -3, np.deg2rad(-90)])
    #qgoal = np.array([0, 0, np.deg2rad(0)])
    
    # Lembrar de habilitar o 'Real-time mode'    
    # Parar a simulação se estiver executando
    initial_sim_state = sim.getSimulationState()
    if initial_sim_state != 0:
        sim.stopSimulation()
        time.sleep(1)
    
    # Inicia a simulação
    sim.startSimulation()
    sim.step()

    # Frame que representa o Goal
    goalFrame = sim.getObject('/Goal')
    sim.setObjectPosition(goalFrame, [qgoal[0], qgoal[1], 0], sim.handle_world)
    sim.setObjectOrientation(goalFrame, [0, 0, qgoal[2]], sim.handle_world)
    
    gain = np.array([[0.1, 0, 0], [0, 0.1, 0], [0, 0, 0.1]])
    #gain = np.array([[0.3, 0, 0], [0, 0.1, 0], [0, 0, 0.1]])
    
    print("Starting robot control loop...")
    while (sim_time := sim.getSimulationTime()) <= 90:
        print(f"Simulation time: {sim_time:.2f} [s]")
        
        dt = sim.getSimulationTimeStep()
                
        pos = sim.getObjectPosition(robotHandle, sim.handle_world)
        ori = sim.getObjectOrientation(robotHandle, sim.handle_world)

        q = np.array([pos[0], pos[1], ori[2]])
        
        error = qgoal - q
        print(f"Error: {np.linalg.norm(error[:2])}")
        
        # Margem aceitável de distância
        if (np.linalg.norm(error[:2]) < 0.05):
            break

        # Controller
        qdot = gain @ error
        
        # Cinemática Inversa
        # w1, w2, w3
        Minv = np.linalg.inv(Rz(q[2]) @ Mdir)
        u = Minv @ qdot
        
        # Enviando velocidades
        sim.setJointTargetVelocity(wheel1, u[0])
        sim.setJointTargetVelocity(wheel2, u[1])
        sim.setJointTargetVelocity(wheel3, u[2]) 
                
        sim.step()

    # Parando o robô
    print("Stopping robot...")
    sim.setJointTargetVelocity(wheel1, 0)
    sim.setJointTargetVelocity(wheel2, 0)
    sim.setJointTargetVelocity(wheel3, 0)
    
except Exception as e:
    print(f"An error occurred: {e}")
    
# Parando a simulação
sim.stopSimulation()

print('Program ended')

Starting robot control loop...
Simulation time: 0.05 [s]
Error: 4.242654235097534
Simulation time: 0.10 [s]
Error: 4.24096601690775
Simulation time: 0.15 [s]
Error: 4.236436618160254
Simulation time: 0.20 [s]
Error: 4.229272918810171
Simulation time: 0.25 [s]
Error: 4.219813948464151
Simulation time: 0.30 [s]
Error: 4.208311835774299
Simulation time: 0.35 [s]
Error: 4.194924776248705
Simulation time: 0.40 [s]
Error: 4.179690683604065
Simulation time: 0.45 [s]
Error: 4.162820687190165
Simulation time: 0.50 [s]
Error: 4.144917916490327
Simulation time: 0.55 [s]
Error: 4.126597878783066
Simulation time: 0.60 [s]
Error: 4.108016584848496
Simulation time: 0.65 [s]
Error: 4.089175932164096
Simulation time: 0.70 [s]
Error: 4.070275394210765
Simulation time: 0.75 [s]
Error: 4.051293040687507
Simulation time: 0.80 [s]
Error: 4.031981643762271
Simulation time: 0.85 [s]
Error: 4.012450441593297
Simulation time: 0.90 [s]
Error: 3.9928132929937483
Simulation time: 0.95 [s]
Error: 3.9731660104054205

Error: 1.963500619163694
Simulation time: 8.10 [s]
Error: 1.9537644790977706
Simulation time: 8.15 [s]
Error: 1.9440763383770812
Simulation time: 8.20 [s]
Error: 1.9344359588838624
Simulation time: 8.25 [s]
Error: 1.9248431036230256
Simulation time: 8.30 [s]
Error: 1.9152975370443035
Simulation time: 8.35 [s]
Error: 1.9057990248453376
Simulation time: 8.40 [s]
Error: 1.8963473339236767
Simulation time: 8.45 [s]
Error: 1.8869422324069371
Simulation time: 8.50 [s]
Error: 1.877583489640611
Simulation time: 8.55 [s]
Error: 1.8682708761635554
Simulation time: 8.60 [s]
Error: 1.8590041636829986
Simulation time: 8.65 [s]
Error: 1.8497831249465688
Simulation time: 8.70 [s]
Error: 1.8406075332626797
Simulation time: 8.75 [s]
Error: 1.8314771636232696
Simulation time: 8.80 [s]
Error: 1.8223917923797153
Simulation time: 8.85 [s]
Error: 1.8133511970096579
Simulation time: 8.90 [s]
Error: 1.8043551556580952
Simulation time: 8.95 [s]
Error: 1.7954034476932015
Simulation time: 9.00 [s]
Error: 1.78649

Error: 0.8842035204787758
Simulation time: 16.10 [s]
Error: 0.8798000475278785
Simulation time: 16.15 [s]
Error: 0.8754185088936071
Simulation time: 16.20 [s]
Error: 0.8710587952703809
Simulation time: 16.25 [s]
Error: 0.8667207978979322
Simulation time: 16.30 [s]
Error: 0.8624044085585879
Simulation time: 16.35 [s]
Error: 0.8581095195745506
Simulation time: 16.40 [s]
Error: 0.8538360238051934
Simulation time: 16.45 [s]
Error: 0.8495838146443784
Simulation time: 16.50 [s]
Error: 0.8453527860177807
Simulation time: 16.55 [s]
Error: 0.8411428323802278
Simulation time: 16.60 [s]
Error: 0.8369538487130529
Simulation time: 16.65 [s]
Error: 0.8327857305214605
Simulation time: 16.70 [s]
Error: 0.8286383738319086
Simulation time: 16.75 [s]
Error: 0.8245116751894979
Simulation time: 16.80 [s]
Error: 0.8204055316553799
Simulation time: 16.85 [s]
Error: 0.8163198408041779
Simulation time: 16.90 [s]
Error: 0.8122545007214134
Simulation time: 16.95 [s]
Error: 0.8082094100009539
Simulation time: 17.

Simulation time: 23.85 [s]
Error: 0.4058113454851493
Simulation time: 23.90 [s]
Error: 0.4037905622111824
Simulation time: 23.95 [s]
Error: 0.4017798427756654
Simulation time: 24.00 [s]
Error: 0.3997791370527636
Simulation time: 24.05 [s]
Error: 0.39778839516638814
Simulation time: 24.10 [s]
Error: 0.39580756748895896
Simulation time: 24.15 [s]
Error: 0.3938366046401552
Simulation time: 24.20 [s]
Error: 0.3918754574856886
Simulation time: 24.25 [s]
Error: 0.389924077136074
Simulation time: 24.30 [s]
Error: 0.38798241494540914
Simulation time: 24.35 [s]
Error: 0.3860504225101581
Simulation time: 24.40 [s]
Error: 0.38412805166794783
Simulation time: 24.45 [s]
Error: 0.3822152544963603
Simulation time: 24.50 [s]
Error: 0.38031198331174165
Simulation time: 24.55 [s]
Error: 0.37841819066800597
Simulation time: 24.60 [s]
Error: 0.37653382935545043
Simulation time: 24.65 [s]
Error: 0.37465885239958624
Simulation time: 24.70 [s]
Error: 0.37279321305995483
Simulation time: 24.75 [s]
Error: 0.37

Error: 0.1862637398207342
Simulation time: 31.70 [s]
Error: 0.18533630837228224
Simulation time: 31.75 [s]
Error: 0.1844134954401742
Simulation time: 31.80 [s]
Error: 0.18349527802397683
Simulation time: 31.85 [s]
Error: 0.18258163323780655
Simulation time: 31.90 [s]
Error: 0.18167253830976546
Simulation time: 31.95 [s]
Error: 0.1807679705813738
Simulation time: 32.00 [s]
Error: 0.17986790750699902
Simulation time: 32.05 [s]
Error: 0.17897232665330115
Simulation time: 32.10 [s]
Error: 0.1780812056986642
Simulation time: 32.15 [s]
Error: 0.1771945224326489
Simulation time: 32.20 [s]
Error: 0.17631225475543322
Simulation time: 32.25 [s]
Error: 0.1754343806772647
Simulation time: 32.30 [s]
Error: 0.1745608783179053
Simulation time: 32.35 [s]
Error: 0.17369172590609436
Simulation time: 32.40 [s]
Error: 0.1728269017790008
Simulation time: 32.45 [s]
Error: 0.17196638438168282
Simulation time: 32.50 [s]
Error: 0.17111015226655835
Simulation time: 32.55 [s]
Error: 0.17025818409285315
Simulatio

Simulation time: 39.45 [s]
Error: 0.08550191338889321
Simulation time: 39.50 [s]
Error: 0.08507626288695799
Simulation time: 39.55 [s]
Error: 0.08465273206354812
Simulation time: 39.60 [s]
Error: 0.08423131036299039
Simulation time: 39.65 [s]
Error: 0.08381198728217894
Simulation time: 39.70 [s]
Error: 0.08339475237030614
Simulation time: 39.75 [s]
Error: 0.08297959522861424
Simulation time: 39.80 [s]
Error: 0.08256650551012938
Simulation time: 39.85 [s]
Error: 0.08215547291940564
Simulation time: 39.90 [s]
Error: 0.08174648721226914
Simulation time: 39.95 [s]
Error: 0.08133953819555897
Simulation time: 40.00 [s]
Error: 0.08093461572687537
Simulation time: 40.05 [s]
Error: 0.08053170971432691
Simulation time: 40.10 [s]
Error: 0.08013081011628337
Simulation time: 40.15 [s]
Error: 0.07973190694111575
Simulation time: 40.20 [s]
Error: 0.07933499024695416
Simulation time: 40.25 [s]
Error: 0.07894005014144286
Simulation time: 40.30 [s]
Error: 0.0785470767814828
Simulation time: 40.35 [s]
Er

# Configuration (pose) control

In [5]:
# Normalize angle to the range [-pi,pi)
def normalizeAngle(angle):
    return np.mod(angle+np.pi, 2*np.pi) - np.pi

try:
    # Connect to the CoppeliaSim server
    client = RemoteAPIClient()
    sim = client.require("sim")
    sim.setStepping(True)

    # Handle para o ROBÔ
    robotname = 'Pioneer_p3dx'
    robotHandle = sim.getObject('/' + robotname)
    
    # Handle para as juntas das RODAS
    l_wheel = sim.getObject('/' + robotname + '/'+ robotname + '_leftMotor')
    r_wheel = sim.getObject('/' + robotname + '/'+ robotname + '_rightMotor')
    
    # Dados Pioneer
    # https://www.generationrobots.com/media/Pioneer3DX-P3DX-RevA.pdf
    # L = 0.381   # Metros
    # r = 0.0975  # Metros
    
    L = 0.331
    r = 0.09751
    maxv = 1.0
    maxw = np.deg2rad(45)
    
    # Goal configuration (x, y, theta)
    qgoal = np.array([2, -2, np.deg2rad(90)])
    #qgoal = np.array([-2, -4, np.deg2rad(180)])
    
    # Lembrar de habilitar o 'Real-time mode'    
    # Parar a simulação se estiver executando
    initial_sim_state = sim.getSimulationState()
    if initial_sim_state != 0:
        sim.stopSimulation()
        time.sleep(1)
    
    # Inicia a simulação
    sim.startSimulation()
    sim.step()

    # Frame que representa o Goal
    goalFrame = sim.getObject('/Goal')
    sim.setObjectPosition(goalFrame, [qgoal[0], qgoal[1], 0], sim.handle_world)
    sim.setObjectOrientation(goalFrame, [0, 0, qgoal[2]], sim.handle_world)
       
    print("Starting robot control loop...")
    while (sim_time := sim.getSimulationTime()) <= 90:
        print(f"Simulation time: {sim_time:.2f} [s]")
        
        dt = sim.getSimulationTimeStep()
                
        robotPos = sim.getObjectPosition(robotHandle, sim.handle_world)
        robotOri = sim.getObjectOrientation(robotHandle, sim.handle_world)
        robotConfig = np.array([robotPos[0], robotPos[1], robotOri[2]])
        
        dx, dy, dth = qgoal - robotConfig
        
        rho = np.sqrt(dx**2 + dy**2)
        alpha = normalizeAngle(-robotConfig[2] + np.arctan2(dy,dx))
        beta = normalizeAngle(qgoal[2] - np.arctan2(dy,dx))
        
        print(f"Error (rho): {rho}")
        if rho <= .05:
            break
        
        kr = 4 / 10
        ka = 8 / 10
        kb = -1.5 / 10
        
        # Alvo na parte de trás
        if abs(alpha) > np.pi/2:
            kr = -kr       
            
            # Se não ajustar a direção muda
            alpha = normalizeAngle(alpha-np.pi)
            beta = normalizeAngle(beta-np.pi)
        
        v = kr*rho
        w = ka*alpha + kb*beta
        
        # Limit v,w to +/- max
        v = max(min(v, maxv), -maxv)
        w = max(min(w, maxw), -maxw)        
        
        wr = ((2.0*v) + (w*L))/(2.0*r)
        wl = ((2.0*v) - (w*L))/(2.0*r)
        
        # Enviando velocidades
        sim.setJointTargetVelocity(l_wheel, wl)
        sim.setJointTargetVelocity(r_wheel, wr)
        
        sim.step()

    # Parando o robô
    print("Stopping robot...")
    sim.setJointTargetVelocity(l_wheel, 0)
    sim.setJointTargetVelocity(r_wheel, 0)
    
except Exception as e:
    print(f"An error occurred: {e}")
    
# Parando a simulação
sim.stopSimulation()

print('Program ended')

Starting robot control loop...
Simulation time: 0.05 [s]
Error (rho): 7.44421827192636
Simulation time: 0.10 [s]
Error (rho): 7.43961116315956
Simulation time: 0.15 [s]
Error (rho): 7.431958165582844
Simulation time: 0.20 [s]
Error (rho): 7.42123325768483
Simulation time: 0.25 [s]
Error (rho): 7.407427099001651
Simulation time: 0.30 [s]
Error (rho): 7.39061040609651
Simulation time: 0.35 [s]
Error (rho): 7.370810423988557
Simulation time: 0.40 [s]
Error (rho): 7.34808968905095
Simulation time: 0.45 [s]
Error (rho): 7.323902715779521
Simulation time: 0.50 [s]
Error (rho): 7.298944415409534
Simulation time: 0.55 [s]
Error (rho): 7.273858167087652
Simulation time: 0.60 [s]
Error (rho): 7.249085871718174
Simulation time: 0.65 [s]
Error (rho): 7.224647183488167
Simulation time: 0.70 [s]
Error (rho): 7.201030196806673
Simulation time: 0.75 [s]
Error (rho): 7.178933546880161
Simulation time: 0.80 [s]
Error (rho): 7.158523701857435
Simulation time: 0.85 [s]
Error (rho): 7.139795220608402
Simul

Error (rho): 2.2635611686740305
Simulation time: 7.30 [s]
Error (rho): 2.2264439724754608
Simulation time: 7.35 [s]
Error (rho): 2.189849728464315
Simulation time: 7.40 [s]
Error (rho): 2.154158063964221
Simulation time: 7.45 [s]
Error (rho): 2.1191878834278244
Simulation time: 7.50 [s]
Error (rho): 2.0848578808656053
Simulation time: 7.55 [s]
Error (rho): 2.0511704619178444
Simulation time: 7.60 [s]
Error (rho): 2.018120075471397
Simulation time: 7.65 [s]
Error (rho): 1.985682480938157
Simulation time: 7.70 [s]
Error (rho): 1.9538178356987654
Simulation time: 7.75 [s]
Error (rho): 1.922440623884285
Simulation time: 7.80 [s]
Error (rho): 1.8915780631266081
Simulation time: 7.85 [s]
Error (rho): 1.8612624648653644
Simulation time: 7.90 [s]
Error (rho): 1.8315240399657668
Simulation time: 7.95 [s]
Error (rho): 1.8023505465004028
Simulation time: 8.00 [s]
Error (rho): 1.7736758127190448
Simulation time: 8.05 [s]
Error (rho): 1.7457246769089814
Simulation time: 8.10 [s]
Error (rho): 1.7182

Error (rho): 0.2263801624411244
Simulation time: 14.30 [s]
Error (rho): 0.22228823906187972
Simulation time: 14.35 [s]
Error (rho): 0.21826237555556682
Simulation time: 14.40 [s]
Error (rho): 0.2143073635219724
Simulation time: 14.45 [s]
Error (rho): 0.21041724565984676
Simulation time: 14.50 [s]
Error (rho): 0.20658854186128425
Simulation time: 14.55 [s]
Error (rho): 0.20283068503975324
Simulation time: 14.60 [s]
Error (rho): 0.19911779760662954
Simulation time: 14.65 [s]
Error (rho): 0.19547868453782816
Simulation time: 14.70 [s]
Error (rho): 0.191898391560784
Simulation time: 14.75 [s]
Error (rho): 0.18837558772766722
Simulation time: 14.80 [s]
Error (rho): 0.18494496721765616
Simulation time: 14.85 [s]
Error (rho): 0.181577156893061
Simulation time: 14.90 [s]
Error (rho): 0.17836575880006794
Simulation time: 14.95 [s]
Error (rho): 0.17518640322803147
Simulation time: 15.00 [s]
Error (rho): 0.1719626421088211
Simulation time: 15.05 [s]
Error (rho): 0.16880615611567779
Simulation tim

# Position control (\[De Luca e Oriolo, 1994\])

In [ ]:
try:
    # Connect to the CoppeliaSim server
    client = RemoteAPIClient()
    sim = client.require("sim")
    sim.setStepping(True)

    # Handle para o ROBÔ
    robotname = 'Pioneer_p3dx'
    robotHandle = sim.getObject('/' + robotname)
    
    # Handle para as juntas das RODAS
    l_wheel = sim.getObject('/' + robotname + '/'+ robotname + '_leftMotor')
    r_wheel = sim.getObject('/' + robotname + '/'+ robotname + '_rightMotor')
    
    # Dados Pioneer
    # https://www.generationrobots.com/media/Pioneer3DX-P3DX-RevA.pdf
    # L = 0.381   # Metros
    # r = 0.0975  # Metros
    
    L = 0.331
    r = 0.09751
    maxv = 1.0
    maxw = np.deg2rad(45)
          
    # Goal position (x, y)
    pgoal = np.array([2.0, -2.0])
    # pgoal = np.array([0.0, 0.0])
    
    # Lembrar de habilitar o 'Real-time mode'    
    # Parar a simulação se estiver executando
    initial_sim_state = sim.getSimulationState()
    if initial_sim_state != 0:
        sim.stopSimulation()
        time.sleep(1)
    
    # Inicia a simulação
    sim.startSimulation()
    sim.step()

    # Frame que representa o Goal
    goalFrame = sim.getObject('/Goal')
    sim.setObjectPosition(goalFrame, [pgoal[0], pgoal[1], 0], sim.handle_world)    
    
    print("Starting robot control loop...")
    while (sim_time := sim.getSimulationTime()) <= 90:
        print(f"Simulation time: {sim_time:.2f} [s]")
        
        dt = sim.getSimulationTimeStep()
                
        robotPos = sim.getObjectPosition(robotHandle, sim.handle_world)
        robotOri = sim.getObjectOrientation(robotHandle, sim.handle_world)
        robotConfig = np.array([robotPos[0], robotPos[1], robotOri[2]])
        
        dx, dy = pgoal - robotConfig[:2]
        
        # Apenas para interromper o loop
        rho = np.sqrt(dx**2 + dy**2)
        print(f"Error (rho): {rho}")
        if rho <= .05:
            break           
            
        kr = 1
        kt = 2
        
        v = kr*(dx*np.cos(robotConfig[2]) + dy*np.sin(robotConfig[2]))
        w = kt*(np.arctan2(dy,dx) - robotConfig[2])
        
        # Limit v,w to +/- max
        v = max(min(v, maxv), -maxv)
        w = max(min(w, maxw), -maxw)        
        
        wr = ((2.0*v) + (w*L))/(2.0*r)
        wl = ((2.0*v) - (w*L))/(2.0*r)
        
        # Enviando velocidades
        sim.setJointTargetVelocity(l_wheel, wl)
        sim.setJointTargetVelocity(r_wheel, wr)
        
        sim.step()

    # Parando o robô
    print("Stopping robot...")
    sim.setJointTargetVelocity(l_wheel, 0)
    sim.setJointTargetVelocity(r_wheel, 0)
    
except Exception as e:
    print(f"An error occurred: {e}")
    
# Parando a simulação
sim.stopSimulation()

print('Program ended')

# Para praticar

- Considerando um robô holonômico (Robotino, youBot), faça o controle de forma que ele consiga realizar um trajeto no formato de uma figura geométrica (e.g., triângulo, quadrado, etc).
- Considerando um robô holonômico (Robotino, youBot), faça o controle de forma que ele consiga seguir uma curva genérica.
- Considerando diferentes tipos de robôs (holonômico e não-holonômico), compare os diferentes trajetos necessários para visitar um conjunto de pontos aleatórios no ambiente.